# NLP Fuzzy matching protype code

In [10]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import string
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

In [11]:
# Columns we are interested in
cols = ["Product", "Platform"]

#Read data
pdf = pd.read_csv("/Users/maxnethercott/Documents/data_science_projects/nlp_sentence_matching/fuzzy_prototype/ms_security_updates_csv.csv")
pdf = pdf[cols]
pdf.columns = pdf.columns.str.lower()
pdf = pdf.fillna("")

#Lower all pandas column values
pdf = pdf.apply(lambda x: x.astype(str).str.lower())

pdf['document'] = pdf[pdf.columns].agg('.'.join, axis=1)
pdf.drop_duplicates(inplace=True)


In [12]:
doc_list = pdf["document"].tolist()
tokenized_sents = [word_tokenize(i) for i in doc_list]

#Filter out stopwords and punctuation...
stop_words = set(nltk.corpus.stopwords.words("english"))

#Strip out stop words
out = [[w for w in x if not w in list(stop_words)] for x in tokenized_sents]
#Strip out punctuation
out = [[s.translate(str.maketrans("", "", string.punctuation)) for s in x] for x in out]

#Use porter stemmer to get root word meaning... Seems to have a positive effect on accuracy
out = [[porter.stem(word) for word in x if word != '']  for x in out]

#Detokenize
detokenized_sents = [TreebankWordDetokenizer().detokenize(i) for i in out]
data = list(set(detokenized_sents))

In [13]:
# Ready for model build
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
max_epochs = 50
vec_size = 150
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                min_count=1,
                dm =0,
                window =1,
                workers=4,
                sample = 0)

model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
Model Saved


In [28]:
def output_sentences(most_similar, data):
    for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(most_similar)//2), ('LEAST', len(most_similar) - 1)]:
      print(u'%s %s: %s\n' % (label, most_similar[index][1], data[int(most_similar[index][0])]))


model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("windows 2016".lower())
test_data = [porter.stem(word) for word in test_data]
v1 = model.infer_vector(test_data)
#print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.dv.most_similar([v1])
print(similar_doc)

# to print similar sentences
output_sentences(similar_doc, data) 

# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
#print(model.dv['1'])

[('111', 0.9245244860649109), ('113', 0.878325879573822), ('2', 0.7650336623191833), ('135', 0.7368009090423584), ('93', 0.7337137460708618), ('100', 0.7236089706420898), ('80', 0.7106636762619019), ('83', 0.7092891335487366), ('28', 0.7071241736412048), ('148', 0.7030264735221863)]
MOST 0.9245244860649109: window server 2016

SECOND-MOST 0.878325879573822: window server 2016 server core instal

MEDIAN 0.7236089706420898: microsoft offic 2016 64bit edit

LEAST 0.7030264735221863: microsoft exchang server 2016 cumul updat 21

